In [ ]:
import json
import pytorch_lightning as pl



from modelAB import *
from modelCD import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_TOKEN = '<pad>'
EN_TRAIN_PATH = "./../../data/EN/train.json"
EN_DEV_PATH = "./../../data/EN/dev.json"
BERT_PATH = "./../../model/bert-base-cased"
#BERT_PATH = "./model/bert-base-cased"
VERBATLAS_PATH = "./VerbAtlas/VerbAtlas"


In [2]:
def read_dataset(path: str):
    with open(path) as f:
        dataset = json.load(f)

    sentences, labels = {}, {}
    for sentence_id, sentence in dataset.items():
        sentence_id = sentence_id
        sentences[sentence_id] = {
            "words": sentence["words"],
            "lemmas": sentence["lemmas"],
            "pos_tags": sentence["pos_tags"],
            "dependency_heads": [int(head) for head in sentence["dependency_heads"]],
            "dependency_relations": sentence["dependency_relations"],
            "predicates": sentence["predicates"],
        }

        labels[sentence_id] = {
            "predicates": sentence["predicates"],
            "roles": {int(p): r for p, r in sentence["roles"].items()}
            if "roles" in sentence
            else dict(),
        }

    return sentences, labels

In [4]:
#'''
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_f1',  # the value that will be evaluated to activate the early stopping of the model.
    patience=5,
    # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.
    verbose=True,  # whether to log or not information in the console.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
)

check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_f1',  # the value that we want to use for model selection.
    verbose=True,  # whether to log or not information in the console.
    save_top_k=1,  # the number of checkpoints we want to store.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
    filename='modelAB_{epoch}-{val_f1:.4f}'
    # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.
)

ab_dm = PredicatesDataModule(
    verbAtlas_path=VERBATLAS_PATH,
    data_train_path=EN_TRAIN_PATH,
    data_dev_path=EN_DEV_PATH,
    data_test_path=EN_DEV_PATH,
    batch_size=8
)

ab_classifier = AB_Model(language="en", hidden2=200, lstm_layers=1, bidirectional=False, p=0.5).to(device)

# the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=20,  # maximum number of epochs.
    gpus=1,  # the number of gpus we have at our disposal.
    callbacks=[early_stopping, check_point_callback]  # the callback we want our trainer to use.
)

# and finally we can let the "trainer" fit the amazon reviews classifier.
trainer.fit(model=ab_classifier, datamodule=ab_dm)

model_path = "../../model/modelAB.ckpt"
#classifier = AB_Model.load_from_checkpoint(model_path, language="en").to(device)




sentences, labels = read_dataset(EN_DEV_PATH)

for idx, key in enumerate(sentences):
    prediction = ab_classifier.predict(sentences[key])
    lab = labels[key]["predicates"]
    print("PREDICTED", len(prediction), prediction)
    print("GROUND TRUTH", len(lab), lab)
#'''

NUM_CLASSES  434


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.5.crossattention.

432
432


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | classifier | Sequential       | 241 K 
1 | loss_fn    | CrossEntropyLoss | 0     
2 | f1         | F1Score          | 0     
3 | bert       | BertModel        | 136 M 
------------------------------------------------
136 M     Trainable params
0         Non-trainable params
136 M     Total params
547.673   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Metric val_f1 improved by 0.000 >= min_delta = 0.0. New best score: 0.976
Epoch 18, global step 13071: val_f1 reached 0.97633 (best 0.97633), saving model to "/home/michele/PycharmProjects/nlp2022-hw2-main/hw2/stud/lightning_logs/version_0/checkpoints/modelAB_epoch=18-val_f1=0.9763.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Metric val_f1 improved by 0.000 >= min_delta = 0.0. New best score: 0.977
Epoch 19, global step 13759: val_f1 reached 0.97672 (best 0.97672), saving model to "/home/michele/PycharmProjects/nlp2022-hw2-main/hw2/stud/lightning_logs/version_0/checkpoints/modelAB_epoch=19-val_f1=0.9767.ckpt" as top 1


432


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)

In [3]:
#'''
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_f1',  # the value that will be evaluated to activate the early stopping of the model.
    patience=5,  # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.
    verbose=True,  # whether to log or not information in the console.
    mode='max', # wheter we want to maximize (max) or minimize the "monitor" value.
)

check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_f1',  # the value that we want to use for model selection.
    verbose=True,  # whether to log or not information in the console.
    save_top_k=1,  # the number of checkpoints we want to store.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
    filename='model_CD{epoch}-{val_f1:.4f}'  # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.
)


cd_dm = SentencesDataModule(
    data_train_path=EN_TRAIN_PATH,
    data_dev_path=EN_DEV_PATH,
    data_test_path=EN_DEV_PATH,
    batch_size=16
)
cd_classifier = CD_Model(language="en",hidden1=768,lstm_layers=1,bidirectional=False)
# the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=20,  # maximum number of epochs.
    gpus=1,  # the number of gpus we have at our disposal.
    callbacks=[early_stopping, check_point_callback]  # the callback we want our trainer to use.
)
# and finally we can let the "trainer" fit the amazon reviews classifier.
#trainer.fit(model=cd_classifier, datamodule=cd_dm)
model_path = "../../model/modelCD.ckpt"
classifier = CD_Model.load_from_checkpoint(model_path,language="en").to(device)



sentences,labels = read_dataset(EN_DEV_PATH)


for idx,key in enumerate(sentences):
    prediction = cd_classifier.predict(sentences[key])["roles"]
    lab = labels[key]["roles"]
    print("PREDICTED",prediction)
    print("GROUND TRUTH",lab)

#'''

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.2.crossattention.

Validation sanity check: 0it [00:00, ?it/s]

/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 whic

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Metric val_f1 improved by 0.001 >= min_delta = 0.0. New best score: 0.948
Epoch 6, global step 5536: val_f1 reached 0.94848 (best 0.94848), saving model to "/home/michele/PycharmProjects/nlp2022-hw2-main/hw2/stud/lightning_logs/version_2/checkpoints/model_CDepoch=6-val_f1=0.9485.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Metric val_f1 improved by 0.001 >= min_delta = 0.0. New best score: 0.949
Epoch 7, global step 6327: val_f1 reached 0.94932 (best 0.94932), saving model to "/home/michele/PycharmProjects/nlp2022-hw2-main/hw2/stud/lightning_logs/version_2/checkpoints/model_CDepoch=7-val_f1=0.9493.ckpt" as top 1
/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)